In [27]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [28]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [29]:
len(words)

32033

In [30]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos)
print(f'{itos=}')
print(f'{vocab_size=}')

itos={1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
vocab_size=27


In [31]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) # 10%
Xte, Yte = build_dataset(words[n2:]) # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [32]:
# MLP
n_embed = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((vocab_size, n_embed), generator=g)
# below is kaiming init
# gain = 5/3 for tanh non-linearity
W1 = torch.randn((n_embed * block_size, n_hidden), generator=g) * (5/3)/((n_embed * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.01
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01 # W2 scaled down significantly
b2 = torch.randn(vocab_size, generator=g) * 0 # b2 reduced to 0

bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))

parameters = [C, W1, b1, W2, b2, bngain, bnbias]

n_parameters = sum(p.nelement() for p in parameters)
print(f'{n_parameters=}')
for p in parameters:
    p.requires_grad = True

n_parameters=12297


In [33]:
max_steps = 100000
batch_size = 32
lossi = []

for i in range(max_steps):
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,),generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X, Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # cancatenate the vectors
    hpreact = embcat @ W1 + b1 # hidden layer pre-activation
    hpreact = bngain * ((hpreact - hpreact.mean(0, keepdim=True)) / hpreact.std(0, keepdim=True)) + bnbias

    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    lr = 0.1 if i < 50000 else 0.01 # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad
        # track stats
    if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

      0/ 100000: 3.3147
  10000/ 100000: 2.1984
  20000/ 100000: 2.3375
  30000/ 100000: 2.4359
  40000/ 100000: 2.0119
  50000/ 100000: 2.2595
  60000/ 100000: 2.4233
  70000/ 100000: 1.9934
  80000/ 100000: 2.2457
  90000/ 100000: 2.0637


In [34]:
# calibrate the batch norm at the end of training

with torch.no_grad():
    # pass the training set through
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    # measure the mean/std over the entire training set
    bnmean = hpreact.mean(0, keepdim = True)
    bnstd = hpreact.std(0, keepdim = True)

In [35]:
@torch.no_grad() # this decorator disables gradient tracking
# this prevents torch from keeping track of computational graph, because we aren't going to do backward pass
def split_loss(split):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    emb = C[x] # (N, block_size, n_embed)
    embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embed)
    hpreact = embcat @ W1 + b1
    # hpreact = bngain * (hpreact - hpreact.mean(0, keepdim=True) / hpreact.std(0, keepdim=True)) + bnbias
    hpreact = bngain * ((hpreact - bnmean) / bnstd) + bnbias
    h = torch.tanh(hpreact) # (N, n_hidden)
    logits = h @ W2 + b2 # (N, vocab_size)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.09417462348938
val 2.124377489089966


In [36]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      hpreact = emb.view(1, -1) @ W1 + b1
      hpreact = bngain * ((hpreact - bnmean) / bnstd) + bnbias
      h = torch.tanh(hpreact)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carmah.
amille.
khi.
mrix.
taty.
salaysie.
rahnee.
delynn.
jarqui.
nellara.
chaiir.
kaleigh.
ham.
join.
quinn.
suli.
malianni.
wate.
madiaryni.
jace.


In [37]:
# conclusion
# loss log

'''
original: 
train 2.1267659664154053
val 2.1697638034820557

fix softmax confidently wrong, reducing logits extremes, by reducing initial size of W2, b2:
train 2.0695769786834717
val 2.131199836730957

fix tanh layer too saturated at init, preventing h taking values at tails of tanh after "first iteration",
by reducing initial size of W1, b1
train 2.0369434356689453
val 2.103416919708252

add a batch norm layer
train 2.0459063053131104
val 2.1086337566375732
'''; 

In [38]:
# what we want is before applying tanh, we need to make sure that 
# hpreact is neither too small or too extreme
# remember below is the tanh backward implementation

'''
def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

focus on this formula - self.grad += (1 - t**2) * out.grad

if t = 0, then during backward pass gradient will pass on to children nodes
if t = 1/-1, then gradient passed will be 0
'''

# so we want to hpreact to be gaussian distribution, values uniformly distributed b/w -1 and 1
# so we perform batch normalisation

# why multiply by bngain and then add bnbias?
# answer - because we want to allow hpreact to remain gaussian but allow it's scaling and movement
# so multiplying by bngain ensures model can scale up or down hpreact, and adding bnbias ensures
# it can move the hpreact left or right according to optimization during the NN training 

"\ndef tanh(self):\n        x = self.data\n        t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)\n        out = Value(t, (self,), 'tanh')\n        def _backward():\n            self.grad += (1 - t**2) * out.grad\n        out._backward = _backward\n        return out\n\nfocus on this formula - self.grad += (1 - t**2) * out.grad\n\nif t = 0, then during backward pass gradient will pass on to children nodes\nif t = 1/-1, then gradient passed will be 0\n"